<img src="./images/DLI_Header.png" style="width: 400px;">

# 3.0 Nemo Microservices Components Deployment


The following components are deployed to support the NeMo microservice functionality:

- Argo Workflows Server: Provides a workflow engine for job orchestration
- NeMo Operator: Manages the lifecycle of custom resources used by NeMo microservices, such as training jobs
- Volcano Admission Service: Validates custom resources such as training jobs
- Volcano Scheduler Service: Provides batch scheduling for high-performance workloads

The following databases support NeMo microservice functionality:

- Milvus: Vector database storage for NeMo Evaluator
- NeMo Customizer Database: Wraps PostgreSQL to store configurations for model customizations.
- NeMo Evaluator Database: Wraps PostgreSQL to store configurations for model evaluations.
- NeMo Entity Store Database: Wraps PostgreSQL to store platform-wide entities such as namespaces, models, and datasets.


## 3.0 Local Lab's Setup

To make this course self-contained, we created a local registry populated with the assets (containers, models and Helm charts) that we will use in the lab.
In order to reproduce it on your own environment, you will need minimal changes to substitute some of the resources with NGC-hosted one's. The private registry is exposed at oci://registry:5000/. Let's create global variables for the IP of the local registry and the Kubernetes Control-plane IP. Those variables will be use it in the entire notebook. # Create Global variables


In [ ]:
# get the Kubernetes Controlplane IP
minikube_ip=!minikube ip
minikube_ip=minikube_ip[0]
minikube_ip

<div class="alert alert-block alert-warning">

Add the git Username
</div>

In [ ]:
print("Please enter your Git username:")
git_username = input()

# Add assertions to validate the variables
assert git_username and git_username.strip(), "GitHub username cannot be empty"


In [ ]:
git_repo_name="llmops-nvidia"
git_base_url="github.com"
applications_base_dir = "llmops-nvidia/applications"
secrets_base_dir = "llmops-nvidia/secrets"
ingress_base_dir = "llmops-nvidia/ingress"

In [ ]:
git_repo_url=f"git@{git_base_url}:{git_username}/{git_repo_name}.git" 
git_repo_url_ssh=f"ssh://git@{git_base_url}/{git_username}/{git_repo_name}.git"
commit_name=git_username 
commit_email=f"{git_username}@llmops-nvidia"
print(git_repo_url)

## 3.1 Add different Repos 

We need to add various helm repositories which are needed for deployment of nemo microservices

In [ ]:
helm_public_repositories = [
    {
        "repository_name": "prometheus-community",
        "repository_url": "https://prometheus-community.github.io/helm-charts",
        "enable_oci": "false"
    },
    {
        "repository_name": "nvidia",
        "repository_url": "https://helm.ngc.nvidia.com/nvidia",
        "enable_oci": "false"
    },
    {
        "repository_name": "zilliztech",
        "repository_url": "https://zilliztech.github.io/milvus-helm",
        "enable_oci": "false"
    },
    {
        "repository_name": "argo-helm",
        "repository_url": "https://argoproj.github.io/argo-helm",
        "enable_oci": "false"
    },
    {
        "repository_name": "community-charts",
        "repository_url": "https://community-charts.github.io/helm-charts",
        "enable_oci": "false"
    },
    {
        "repository_name": "bitnamicharts",
        "repository_url": "registry-1.docker.io/bitnamicharts",
        "enable_oci": "true"
    },
    {
        "repository_name": "volcano-sh",
        "repository_url": "https://volcano-sh.github.io/helm-charts",
        "enable_oci": "false"
    },
    {
        "repository_name": "langflow",
        "repository_url": "https://langflow-ai.github.io/langflow-helm-charts",
        "enable_oci": "false"
    }
]

In [ ]:
helm_private_repositories = [
    {
      "repository_name":  "nemo-microservices",
      "repository_url":  "docker-registry.registry.svc.cluster.local/nvidia/nemo-microservices/charts",
      "username":  "",
      "password": "",
      "enable_oci": "true"
    }
]

### 3.1.1 Add public helm chart repositories in ArgoCD

In [ ]:
import os
# Ensure the directory exists
output_dir = "argocd_configs/public_repos"
os.makedirs(output_dir, exist_ok=True)

# Loop over repositories and create YAML files
for repo in helm_public_repositories:
    yaml_content = f"""apiVersion: v1
kind: Secret
metadata:
  name: {repo["repository_name"]}
  namespace: argocd
  labels:
    argocd.argoproj.io/secret-type: repository
stringData:
  url: {repo["repository_url"]}
  name: {repo["repository_name"]}
  type: helm
  enableOCI: "{repo["enable_oci"]}"
"""
    file_path = os.path.join(output_dir, f"{repo['repository_name']}_repo.yaml")
    with open(file_path, "w") as f:
        f.write(yaml_content)


In [ ]:
!ls -l argocd_configs/public_repos

In [ ]:
!kubectl apply -f argocd_configs/public_repos

### 3.1.2 Add private helm chart repositories in ArgoCD

In [ ]:
import os
# Ensure the directory exists
output_dir = "argocd_configs/private_repos"
os.makedirs(output_dir, exist_ok=True)

# Loop over repositories and create YAML files
for repo in helm_private_repositories:
    yaml_content = f"""apiVersion: v1
kind: Secret
metadata:
  name: {repo["repository_name"]}
  namespace: argocd
  labels:
    argocd.argoproj.io/secret-type: repository
stringData:
  url: {repo["repository_url"]}
  name: {repo["repository_name"]}
  username: "{repo["username"]}"
  password: "{repo["password"]}"
  type: helm
  enableOCI: "{repo["enable_oci"]}"
  insecure: "true"
"""
    file_path = os.path.join(output_dir, f"{repo['repository_name']}_repo.yaml")
    with open(file_path, "w") as f:
        f.write(yaml_content)


In [ ]:
!ls -l argocd_configs/private_repos

In [ ]:
!kubectl apply -f argocd_configs/private_repos

### 3.1.3 Check Argo CD UI for the repositories

[Open ArgoCD!](/argocd/settings/repos)

<center><img src="./images-dli/repos-added-argocd.png" style="width: 1000px;"></center>

It should show successful for all the 9 repositories

## 3.2 Create Image and Model PULL Secrets YAML file

In [ ]:
os.makedirs(f"{secrets_base_dir}/nvcr", exist_ok=True)

In [ ]:
nvcr_secrets_yaml = f"""
apiVersion: v1
data:
  .dockerconfigjson: eyJhdXRocyI6eyJudmNyLmlvIjp7InVzZXJuYW1lIjoiJG9hdXRodG9rZW4iLCJwYXNzd29yZCI6ImR1bW15IiwiYXV0aCI6IkpHOWhkWFJvZEc5clpXNDZaSFZ0YlhrPSJ9fX0=
kind: Secret
metadata:
  name: nvcrimagepullsecret
type: kubernetes.io/dockerconfigjson
---
apiVersion: v1
data:
  .dockerconfigjson: eyJhdXRocyI6eyJudmNyLmlvIjp7InVzZXJuYW1lIjoiJG9hdXRodG9rZW4iLCJwYXNzd29yZCI6ImR1bW15IiwiYXV0aCI6IkpHOWhkWFJvZEc5clpXNDZaSFZ0YlhrPSJ9fX0=
kind: Secret
metadata:
  name: gitlab-imagepull
type: kubernetes.io/dockerconfigjson
---
apiVersion: v1
data:
  NGC_CLI_API_KEY: ZHVtbXk=
  NGC_API_KEY: ZHVtbXk=
kind: Secret
metadata:
  name: ngc-api
type: Opaque
"""
with open(f"{secrets_base_dir}/nvcr/secret.yaml", "w") as f:
    f.write(nvcr_secrets_yaml)


In [ ]:
!git config --global user.email $commit_email
!git config --global user.name $commit_user
!cd llmops-nvidia/ && git add . && git commit -m "add nvcr secrets" && git push

## 3.3 PostgreSQL

NeMo Customizer Database: Wraps PostgreSQL to store configurations for model customizations.
NeMo Evaluator Database: Wraps PostgreSQL to store configurations for model evaluations.
NeMo Entity Store Database: Wraps PostgreSQL to store platform-wide entities such as namespaces, models, and datasets.

### 3.3.1 Create Secrets YAML file

In [ ]:
os.makedirs(f"{secrets_base_dir}/postgres", exist_ok=True)

In [ ]:
postgres_secrets_yaml = f"""
apiVersion: v1
data:
  postgresql-admin-password: UXVYMitQMTZTUGM3  
  postgresql-user-password: UXVYMitQMTZTUGM3  
  postgresql-replocation-password: UXVYMitQMTZTUGM3  
kind: Secret
metadata:
  name: existing-postgres-auth-secret
type: Opaque

"""
with open(f"{secrets_base_dir}/postgres/secret.yaml", "w") as f:
    f.write(postgres_secrets_yaml)


### 3.3.2 Create PostgreSQL ArgoCD Application YAML file

In [ ]:
os.makedirs(f"{applications_base_dir}/postgres", exist_ok=True)

In [ ]:
postgres_application_yaml = f"""
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: postgresql
  namespace: argocd
spec:
  project: default
  sources:
    - chart: postgresql
      repoURL: 'registry-1.docker.io/bitnamicharts'
      targetRevision: "15.5.20"
      helm:
        releaseName: postgresql
        valuesObject:
          global:
            defaultStorageClass: standard
            postgresql:
              auth:
                username: "nemo-user"
                existingSecret: existing-postgres-auth-secret
                secretKeys:
                  adminPasswordKey: postgresql-admin-password
                  userPasswordKey: postgresql-user-password
                  replicationPasswordKey: postgresql-replocation-password
          auth:
            enablePostgresUser: true
            username: "nemo-user"
            existingSecret: "existing-postgres-auth-secret"
            secretKeys:
              adminPasswordKey: postgresql-admin-password
              userPasswordKey: postgresql-user-password
              replicationPasswordKey: postgresql-replocation-password
          architecture: standalone
          primary:
            service:
              ports:
                postgresql: 5432  
            initdb:
              scripts:
                create_db.sh: |
                  #!/bin/bash
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "CREATE DATABASE \\"gitea-database\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "CREATE DATABASE \\"api-database\\"" 
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "CREATE DATABASE \\"finetuning\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "CREATE DATABASE \\"gateway\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "CREATE DATABASE \\"evaluation\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "GRANT ALL PRIVILEGES ON DATABASE \\"gitea-database\\" to \\"nemo-user\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "GRANT ALL PRIVILEGES ON DATABASE \\"api-database\\" to \\"nemo-user\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "GRANT ALL PRIVILEGES ON DATABASE \\"postgres\\" to \\"nemo-user\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "GRANT ALL PRIVILEGES ON DATABASE \\"finetuning\\" to \\"nemo-user\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "GRANT ALL PRIVILEGES ON DATABASE \\"gateway\\" to \\"nemo-user\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "GRANT ALL PRIVILEGES ON DATABASE \\"evaluation\\" to \\"nemo-user\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "ALTER DATABASE \\"gitea-database\\" OWNER TO \\"nemo-user\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "ALTER DATABASE \\"api-database\\" OWNER TO \\"nemo-user\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "ALTER DATABASE \\"finetuning\\" OWNER TO \\"nemo-user\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "ALTER DATABASE \\"postgres\\" OWNER TO \\"nemo-user\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "ALTER DATABASE \\"gateway\\" OWNER TO \\"nemo-user\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "ALTER DATABASE \\"evaluation\\" OWNER TO \\"nemo-user\\""
                  PGPASSWORD=$POSTGRES_POSTGRES_PASSWORD psql -U postgres <<< "GRANT ALL ON SCHEMA public to \\"nemo-user\\""

    - repoURL: '{git_repo_url_ssh}'
      path: secrets/postgres
      targetRevision: main
      directory:
        recurse: true    
  destination:
    server: 'https://kubernetes.default.svc'
    namespace: postgresql
  syncPolicy:
    syncOptions:
    - Validate=false
    - CreateNamespace=true
    automated:
      prune: true
      selfHeal: true
      allowEmpty: false
"""
with open(f"{applications_base_dir}/postgres/app.yaml", "w") as f:
    f.write(postgres_application_yaml)

### 3.3.3 Commit the added files

In [ ]:
!git config --global user.email $commit_email
!git config --global user.name $commit_user
!cd llmops-nvidia/ && git add . && git commit -m "add postgres" && git push

### 3.3.4 Sync vi UI

[Open ArgoCD!](/argocd/applications)

As we have commit our code to Git, argocd usually sync automatically after every 5 minutes. But we can force it to sync either via UI or CLI. 

Here we use UI to sync. 

Click on the sycn button on the `argocd-components` application as shown in diagram. You will see `postgresql` application shown up

<center><img src="./images-dli/sync-apps.png" style="width: 435px;"></center>

### 3.3.5 Check applications via UI

[Open ArgoCD!](/argocd/applications)

<center><img src="./images-dli/argocd_apps_postgres.png" style="width: 1000px;"></center>

It should show successful 3 applications

## 3.4 MinIO (Object Store)

It is used by Nemo Datastore to store objects. 

### 3.4.1 Create Secrets YAML file

In [ ]:
os.makedirs(f"{secrets_base_dir}/minio", exist_ok=True)

In [ ]:
minio_secrets_yaml = f"""
apiVersion: v1
data:
  username: YWRtaW4=
  password: UXVYMitQMTZTUGM3
kind: Secret
metadata:
  name: existing-minio-auth-secret
type: Opaque
"""
with open(f"{secrets_base_dir}/minio/secret.yaml", "w") as f:
    f.write(minio_secrets_yaml)


### 3.4.2 Create Ingress YAML file

In [ ]:
os.makedirs(f"{ingress_base_dir}/minio", exist_ok=True)

In [ ]:
minio_secrets_yaml = f"""
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: minio-ingress
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /
spec:
  rules:
    - host: minio.local
      http:
        paths:
          - path: /
            pathType: Prefix
            backend:
              service:
                name: minio-client 
                port:
                  number: 9000
"""
with open(f"{ingress_base_dir}/minio/ingress.yaml", "w") as f:
    f.write(minio_secrets_yaml)

### 3.4.3 Create Application YAML file

In [ ]:
os.makedirs(f"{applications_base_dir}/minio", exist_ok=True)

In [ ]:
minio_application_yaml = f"""
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: minio
  namespace: argocd
spec:
  project: default
  sources:
    - chart: minio
      repoURL: 'registry-1.docker.io/bitnamicharts'
      targetRevision: "14.6.33"
      helm:
        releaseName: minio-client
        valuesObject:
          global:
            defaultStorageClass: standard
          auth:
            existingSecret: "existing-minio-auth-secret"
            rootUserSecretKey: "username"
            rootPasswordSecretKey: "password"
            mode: standalone
            persistence:
              enabled: true
              size: 20Gi
              storageClass: "standard"
              annotations:
                helm.sh/resource-policy: keep
    - repoURL: '{git_repo_url_ssh}'
      path: secrets/minio
      targetRevision: main
      directory:
        recurse: true   
    - repoURL: '{git_repo_url_ssh}'
      path: ingress/minio
      targetRevision: main
      directory:
        recurse: true  
  destination:
    server: 'https://kubernetes.default.svc'
    namespace: minio
  syncPolicy:
    syncOptions:
    - Validate=false
    - CreateNamespace=true
    automated:
      prune: true
      selfHeal: true
      allowEmpty: false
"""
with open(f"{applications_base_dir}/minio/app.yaml", "w") as f:
    f.write(minio_application_yaml)

### 3.4.4 Commit the added files

In [ ]:
!git config --global user.email $commit_email
!git config --global user.name $commit_user
!cd llmops-nvidia/ && git add . && git commit -m "add minio" && git push

### 3.4.5 Sync vi UI

[Open ArgoCD!](/argocd/applications)

As we have commit our code to Git, argocd usually sync automatically after every 5 minutes. But we can force it to sync either via UI or CLI. 

Here we use UI to sync. 

Click on the sycn button on the `argocd-components` application as shown in diagram. You will see `minio` application shown up

<center><img src="./images-dli/sync-apps.png" style="width: 435px;"></center>

## 3.5 Milvus (Vector DB)

### 3.5.1 Create Application YAML file

In [ ]:
os.makedirs(f"{applications_base_dir}/milvus", exist_ok=True)

In [ ]:
milvus_application_yaml = f"""
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: milvus
  namespace: argocd
spec:
  project: default
  sources:
    - chart: milvus
      repoURL: 'https://zilliztech.github.io/milvus-helm'
      targetRevision: "4.1.11"
      helm:
        releaseName: milvus
        valuesObject:
          serviceName: milvus
          cluster:
            enabled: false
          etcd:
            enabled: false
          pulsar:
            enabled: false
          minio:
            enabled: false
            tls:
              enabled: false
          standalone:
            persistence:
              enabled: true
              persistentVolumeClaim:
                size: 20Gi
                storageClass: "standard"
            extraEnv:
              - name: LOG_LEVEL
                value: error
          extraConfigFiles:
            user.yaml: |+
              etcd:
                use:
                  embed: true
                data:
                  dir: /var/lib/milvus/etcd
              common:
                storageType: local
  destination:
    server: 'https://kubernetes.default.svc'
    namespace: milvus
  syncPolicy:
    syncOptions:
    - Validate=false
    - CreateNamespace=true
    automated:
      prune: true
      selfHeal: true
      allowEmpty: false
"""
with open(f"{applications_base_dir}/milvus/app.yaml", "w") as f:
    f.write(milvus_application_yaml)

### 3.5.2 Commit the added files

In [ ]:
!git config --global user.email $commit_email
!git config --global user.name $commit_user
!cd llmops-nvidia/ && git add . && git commit -m "add milvus" && git push

### 3.5.3 Sync vi UI
[Open ArgoCD!](/argocd/applications)

As we have commit our code to Git, argocd usually sync automatically after every 5 minutes. But we can force it to sync either via UI or CLI. 

Here we use UI to sync. 

Click on the sycn button on the `argocd-components` application as shown in diagram. You will see `milvus` application shown up

<center><img src="./images-dli/sync-apps.png" style="width: 435px;"></center>

## 3.6 Argo Workflows

### 3.6.1 Create Application YAML file

In [ ]:
os.makedirs(f"{applications_base_dir}/argoworkflows", exist_ok=True)

In [ ]:
argoworkflows_application_yaml = f"""
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: argoworkflows
  namespace: argocd
spec:
  project: default
  sources:
    - repoURL: 'https://argoproj.github.io/argo-helm'
      targetRevision: 0.41.11
      chart: argo-workflows
      helm:
        releaseName: argo-workflows
        valuesObject:
          nameOverride: argo-workflows
          serviceName: argo-workflows-server
          server:
            baseHref: /
            authModes:
              - "server"
            servicePort: 2746
            serviceType: ClusterIP

  destination:
    server: 'https://kubernetes.default.svc'
    namespace: argoworkflows
  syncPolicy:
    syncOptions:
    - Validate=false
    - CreateNamespace=true
    automated:
      prune: true
      selfHeal: true
      allowEmpty: false
"""
with open(f"{applications_base_dir}/argoworkflows/app.yaml", "w") as f:
    f.write(argoworkflows_application_yaml)

### 3.6.2 Commit the added files

In [ ]:
!git config --global user.email $commit_email
!git config --global user.name $commit_user
!cd llmops-nvidia/ && git add . && git commit -m "add argoworkflows" && git push

### 3.6.3 Sync vi UI
[Open ArgoCD!](/argocd/applications)

As we have commit our code to Git, argocd usually sync automatically after every 5 minutes. But we can force it to sync either via UI or CLI. 

Here we use UI to sync. 

Click on the sycn button on the `argocd-components` application as shown in diagram. You will see `argoworkflows` application shown up

<center><img src="./images-dli/sync-apps.png" style="width: 435px;"></center>

## 3.7 MLFlow

MLflow is an open source platform for managing the end-to-end machine learning lifecycle. It tackles four primary functions:

- Tracking experiments to record and compare parameters and results
- Packaging ML code in a reusable, reproducible form
- Managing and deploying models
- Providing a central model registry

<center><img src="./images-dli/mlflow-ui.png" style="width: 800px;"></center>



### 3.7.1 Create Application YAML file

In [ ]:
os.makedirs(f"{applications_base_dir}/mlflow", exist_ok=True)

In [ ]:
mlflow_application_yaml = f"""
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: mlflow
  namespace: argocd
spec:
  project: default
  sources:
    - chart: mlflow
      repoURL: 'registry-1.docker.io/bitnamicharts'
      targetRevision: "1.4.22"
      helm:
        releaseName: mlflow
        valuesObject:
          tracking:
            service:
              type: NodePort
              nodePorts:
                http: "30090"
            auth:
              enabled: false
  destination:
    server: 'https://kubernetes.default.svc'
    namespace: mlflow
  syncPolicy:
    syncOptions:
    - Validate=false
    - CreateNamespace=true
    automated:
      prune: true
      selfHeal: true
      allowEmpty: false
"""
with open(f"{applications_base_dir}/mlflow/app.yaml", "w") as f:
    f.write(mlflow_application_yaml)

### 3.7.2 Commit the added files

In [ ]:
!git config --global user.email $commit_email
!git config --global user.name $commit_user
!cd llmops-nvidia/ && git add . && git commit -m "add mlflow" && git push

### 3.7.3 Sync vi UI
[Open ArgoCD!](/argocd/applications)

As we have commit our code to Git, argocd usually sync automatically after every 5 minutes. But we can force it to sync either via UI or CLI. 

Here we use UI to sync. 

Click on the sycn button on the `argocd-components` application as shown in diagram. You will see `mlflow` application shown up

<center><img src="./images-dli/sync-apps.png" style="width: 435px;"></center>

## 3.8  Volcano.sh

### 3.8.1 Create Application YAML file

In [ ]:
os.makedirs(f"{applications_base_dir}/volcano", exist_ok=True)

In [ ]:
volcano_application_yaml = f"""
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: volcano
  namespace: argocd
spec:
  project: default
  source:
    chart: volcano
    repoURL: 'https://volcano-sh.github.io/helm-charts'
    targetRevision: "1.9.0"
    helm:
      releaseName: volcano
  destination:
    server: 'https://kubernetes.default.svc'
    namespace: volcano
  syncPolicy:
    syncOptions:
    - Validate=false
    - CreateNamespace=true
    automated:
      prune: true
      selfHeal: true
      allowEmpty: false
"""
with open(f"{applications_base_dir}/volcano/app.yaml", "w") as f:
    f.write(volcano_application_yaml)

### 3.8.2 Commit the added files

In [ ]:
!git config --global user.email $commit_email
!git config --global user.name $commit_user
!cd llmops-nvidia/ && git add . && git commit -m "add volcano.sh" && git push

### 3.8.3 Sync vi UI
[Open ArgoCD!](/argocd/applications)

As we have commit our code to Git, argocd usually sync automatically after every 5 minutes. But we can force it to sync either via UI or CLI. 

Here we use UI to sync. 

Click on the sycn button on the `argocd-components` application as shown in diagram. You will see `volcano` application shown up

<center><img src="./images-dli/sync-apps.png" style="width: 435px;"></center>

## 3.9  NIM Operator

The Operator uses the following custom resources:

- nimcaches.apps.nvidia.com: This custom resource enables downloading models from NVIDIA NGC and persisting them on network storage. One advantage to caching a model is that when multiple instances of the same NIM microservice start, the microservices use the single cached model. However, caching is optional. Without caching, each NIM microservice instance downloads a copy of the model when it starts.

- nimservices.apps.nvidia.com: This custom resource represents a NIM microservice. Adding and updating a NIM service resource creates a Kubernetes deployment for the microservice in a namespace. The custom resource supports using a model from an existing NIM cache resource or a persistent volume claim. The custom resource also supports creating a horizontal pod autoscaler, ingress, and service monitor to simplify cluster administration.

- nimpipelines.apps.nvidia.com: This custom resource represents a group of NIM service custom resources.

<center><img src="./images-dli/nim-operator-high-level.png" style="width: 800px;"></center>


### 3.9.1 Create Application YAML file

In [ ]:
os.makedirs(f"{applications_base_dir}/k8s-nim-operator", exist_ok=True)

In [ ]:
nim_operator_application_yaml = f"""
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: k8s-nim-operator
  namespace: argocd
spec:
  project: default
  sources:
    - chart: k8s-nim-operator
      repoURL: 'https://helm.ngc.nvidia.com/nvidia'
      targetRevision: "1.0.1"
      helm:
        releaseName: k8s-nim-operator
  destination:
    server: 'https://kubernetes.default.svc'
    namespace: k8s-nim-operator
  syncPolicy:
    syncOptions:
    - Validate=false
    - CreateNamespace=true
    automated:
      prune: true
      selfHeal: true
      allowEmpty: false
"""
with open(f"{applications_base_dir}/k8s-nim-operator/app.yaml", "w") as f:
    f.write(nim_operator_application_yaml)

### 3.9.2 Commit the added files

In [ ]:
!git config --global user.email $commit_email
!git config --global user.name $commit_user
!cd llmops-nvidia/ && git add . && git commit -m "add nim operator" && git push

### 3.9.3 Sync vi UI
[Open ArgoCD!](/argocd/applications)

As we have commit our code to Git, argocd usually sync automatically after every 5 minutes. But we can force it to sync either via UI or CLI. 

Here we use UI to sync. 

Click on the sycn button on the `argocd-components` application as shown in diagram. You will see `k9s-nim-operator` application shown up

<center><img src="./images-dli/sync-apps.png" style="width: 435px;"></center>

## 3.10  Nemo Operator

### 3.10.1 Create Application YAML file

In [ ]:
os.makedirs(f"{applications_base_dir}/nemo-kubernetes-operator", exist_ok=True)

In [ ]:
nemo_operator_application_yaml = f"""
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: nemo-kubernetes-operator
  namespace: argocd
spec:
  project: default
  sources:
    - chart: nemo-kubernetes-operator
      repoURL: "docker-registry.registry.svc.cluster.local/nvidia/nemo-microservices/charts"
      targetRevision: "0.1.41"
      helm:
        releaseName: nemo-kubernetes-operator
        valuesObject:
          imagePullSecrets:
            - name: nvcrimagepullsecret
          customizerControllerManager:
            enabled: true
            manager:
              image:
                repository: "192.168.49.2:30500/nvidia/nemo-microservices/nemo-kubernetes-operator"
                tag: 25.01
          dmsControllerManager:
            enabled: false
          compoundaiControllerManager:
            enabled: false
    - repoURL: '{git_repo_url_ssh}'
      path: secrets/nvcr
      targetRevision: main
      directory:
        recurse: true  
  destination:
    server: 'https://kubernetes.default.svc'
    namespace: nemo-kubernetes-operator
  syncPolicy:
    syncOptions:
    - Validate=false
    - CreateNamespace=true
    automated:
      prune: true
      selfHeal: true
      allowEmpty: false
"""
with open(f"{applications_base_dir}/nemo-kubernetes-operator/app.yaml", "w") as f:
    f.write(nemo_operator_application_yaml)

### 3.10.2 Commit the added files

In [ ]:
!git config --global user.email $commit_email
!git config --global user.name $commit_user
!cd llmops-nvidia/ && git add . && git commit -m "add nemo operator" && git push

### 3.10.3 Sync vi UI
[Open ArgoCD!](/argocd/applications)

As we have commit our code to Git, argocd usually sync automatically after every 5 minutes. But we can force it to sync either via UI or CLI. 

Here we use UI to sync. 

Click on the sycn button on the `argocd-components` application as shown in diagram. You will see `k8s-nemo-operator` application shown up

<center><img src="./images-dli/sync-apps.png" style="width: 435px;"></center>

### 3.10.4 Check applications via UI

[Open ArgoCD!](/argocd/applications)

It should show successful 10 applications


<center><img src="./images-dli/nemo-microservices-components.png" style="width: 1000px;"></center>



---
<h2 style="color:green;">Congratulations!</h2>

You've completed the third notebook! In this notebook, you have:
- Added various Helm repositories to ArgoCD
- Deployed essential components needed for NeMo Microservices as ArgoCD applications
- Set up the infrastructure required for running NeMo Microservices

Next, you'll learn how to deploy Nemo Microservices via ArgoCD.

Move on to [04_Nemo_Microservices_Deployment.ipynb](04_Nemo_Microservices_Deployment.ipynb)

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>